# Final Project

## TRAC2- Transformer Models (BERT) - base uncased - Data Augmentation with back translation- Evaluate variability

The purpose of this notebook is to evaluate the variability in the results (f1-score) of the models for classification task-A and task-B.

These models use the augmented training data using back translation (6x minority classes).

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Package imports

In [2]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 52.4 MB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
     |████████████████████████████████| 3.3 MB 58.1 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install datasets

     |████████████████████████████████| 287 kB 4.8 MB/s 
     |████████████████████████████████| 125 kB 59.5 MB/s 
     |████████████████████████████████| 243 kB 56.1 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
     |████████████████████████████████| 271 kB 70.1 MB/s 
     |████████████████████████████████| 160 kB 71.0 MB/s 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import load_dataset

from sklearn.preprocessing import label_binarize
from sklearn import metrics

import statistics

## Helper functions

In [5]:
def from_logits_to_labels(pred, task):
    '''
    Returns labels based on predicted logits on labels [CAG,NAG,OAG] for task A. Task B is binary, and 'GEN' represents 
    the positive class.
    Parameters:
    pred: array with model prediction
    task: either 'A' or 'B'
    '''
    index_a = {0:'CAG', 1:'NAG', 2:'OAG'}
    index_b = {0:'GEN', 1:'NGEN'}
    
    if task == 'A':
        highest_prob_class = np.argmax(pred, axis=1)
        labels = np.vectorize(index_a.get)(highest_prob_class.astype(int))
        
    elif task == 'B':
        highest_prob_class = np.argmax(pred, axis=1)
        labels = np.vectorize(index_b.get)(highest_prob_class.astype(int))
    else:
        labels = []
        
    return labels  

In [6]:
def to_one_hot_labels(string_labels):
    '''
    Returns one-hot encoded labels from a multi-class label vector e.g. ['cat', 'dog', 'dog', 'lion', 'cat', ...] 
    Parameters:
    string_labels: 
    '''
    labels = pd.get_dummies(string_labels)
    labels = labels.to_numpy()
    
    return labels

In [7]:
# this is modified to get the prediction as parameter
# to avoid predicting again since inference takes time
def confusion_matrix_plot(pred_labels, true_labels, task, normalize=None):
    '''
    Returns a confusion matrix with a nice format.
    Parameters:
    pred_labels: predicted labels
    true_labels: true labels 
    task: 'A' or 'B'
    normalize: if want to normalize the confusion matrix normalize='true'
    '''
    
    # Create a confusion matrix
    cm = metrics.confusion_matrix(true_labels, pred_labels, normalize=normalize)
    cm = np.around(cm, 2)

    # Plot the confusion matrix
    if task == 'A':
        axis_labels = ['CAG', 'NAG', 'OAG']
    elif task == 'B':
        axis_labels = ['GEN', 'NGEN']

    fig, ax = plt.subplots(figsize=(4,4))
    im = ax.imshow(cm, cmap="Blues")

    # Create the ticks and labels
    ax.set_xticks(np.arange(len(axis_labels)))
    ax.set_yticks(np.arange(len(axis_labels)))
    ax.set_xticklabels(axis_labels)
    ax.set_yticklabels(axis_labels)

    # Axis titles
    plt.ylabel('True label', size=12)
    plt.xlabel('Predicted label', size=12)

    # Loop over data dimensions and create text annotations.
    for i in range(len(axis_labels)):
        for j in range(len(axis_labels)):
            text = ax.text(j, i, cm[i, j],ha="center", va="center", color="dimgrey", size=12)
    
    ax.set_title("Confusion Matrix", size=16, weight="bold")
    fig.tight_layout()
    plt.show()


In [8]:
def loss_accuracy_plots(training_history, xrange, task):
    '''
    Returns plots for loss and accuracy during the training process of a NN.
    Parameters:
    training_history: object that stores the training history of the NN (from model.fit(...))
    xrange: range in x axis
    task: string used for the title in the plot
    '''
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    # loss plot
    ax1.plot(training_history.history['loss'], color='black')
    ax1.plot(training_history.history['val_loss'], color='blue')
    ax1.set_title('Training and validation loss Sub-Task ' + task)
    ax1.legend(['training', 'development'])
    ax1.grid(which='both')
    ax1.set_xticks(np.arange(0, xrange, 2))
    
    # accuracy plot
    ax2.plot(training_history.history['categorical_accuracy'], color='black')
    ax2.plot(training_history.history['val_categorical_accuracy'], color='blue')
    ax2.set_title('Training and validation acccuracy Sub_Task ' + task)
    ax2.legend(['training', 'development'])
    ax2.grid(which='both')
    ax2.set_xticks(np.arange(0, xrange, 2))
    plt.show()
    

## Load data
Load training, development and test datasets.

In [9]:
# Load labels using pandas dataframes

train_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_train_augm_backt.csv')['Sub-task A']
train_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_train_augm_backt.csv')['Sub-task B']
dev_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv')['Sub-task A']
dev_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv')['Sub-task B']
test_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/gold/trac2_eng_gold_a.csv')['Sub-task A']
test_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/gold/trac2_eng_gold_b.csv')['Sub-task B']

In [10]:
# Load text data using Hugging Face datasets
# need to use the split argument even though we are not splitting. If not, data is loaded as DatasetDict
# to load as dataset need to include the split parameter
train_dataset = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/eng/trac2_eng_train_augm_backt.csv', split = 'train[:9373]')
dev_dataset = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv', split = 'train[:1066]')
test_dataset = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/test/trac2_eng_test.csv', split = 'train[:1200]')

Using custom data configuration default-8fa096771488ff92


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8fa096771488ff92/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


Using custom data configuration default-98e2ce082b0629f1


...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-98e2ce082b0629f1/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


Using custom data configuration default-0ba61447556c92da


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0ba61447556c92da/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


## Encode labels

In [11]:
# encode labels Task A- [CAG,NAG,OAG]
train_labels_a_enc = to_one_hot_labels(train_labels_a)
dev_labels_a_enc = to_one_hot_labels(dev_labels_a)
test_labels_a_enc = to_one_hot_labels(test_labels_a)


In [12]:
# encode labels Task B- [GEN, NGEN]
train_labels_b_enc = to_one_hot_labels(train_labels_b)
dev_labels_b_enc = to_one_hot_labels(dev_labels_b)
test_labels_b_enc = to_one_hot_labels(test_labels_b)

## Prepare TensorFlow datasets for BERT

In [13]:
# remove columns to leave only the column with the posts. Column 'Text'
train_dataset = train_dataset.remove_columns(['ID', 'Sub-task B', 'Sub-task A'])
dev_dataset = dev_dataset.remove_columns(['ID', 'Sub-task A', 'Sub-task B'])
test_dataset = test_dataset.remove_columns('ID')

In [14]:
# define a BERT tokenizer
# use the bert-based-uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [15]:
# tokenize the train, development and test data
# Use a max sequence of 150 tokens. Based on EDA this is enough for majority of posts

train_dataset_tok = train_dataset.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)
dev_dataset_tok = dev_dataset.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)
test_dataset_tok = test_dataset.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
# now we can remove the column with the original post from the dataset. We are going to use the result of tokenization for modeling
train_dataset_tok = train_dataset_tok.remove_columns(['Text']).with_format('tensorflow')
dev_dataset_tok = dev_dataset_tok.remove_columns(['Text']).with_format('tensorflow')
test_dataset_tok = test_dataset_tok.remove_columns(['Text']).with_format('tensorflow')

In [17]:
# extract features from tokenizer output: 'input_ids', 'token_type_ids', 'attention_mask'
train_features = {x: train_dataset_tok[x] for x in tokenizer.model_input_names}
dev_features = {x: dev_dataset_tok[x] for x in tokenizer.model_input_names}
test_features = {x: test_dataset_tok[x] for x in tokenizer.model_input_names}

In [18]:
# batch data

batch_size = 16
buffer = len(train_dataset_tok)

# Task A
train_tf_dataset_a = tf.data.Dataset.from_tensor_slices((train_features, train_labels_a_enc)).shuffle(buffer).batch(batch_size)
dev_tf_dataset_a = tf.data.Dataset.from_tensor_slices((dev_features, dev_labels_a_enc)).batch(batch_size)
test_tf_dataset_a = tf.data.Dataset.from_tensor_slices((test_features, test_labels_a_enc)).batch(batch_size)

# Task B
train_tf_dataset_b = tf.data.Dataset.from_tensor_slices((train_features, train_labels_b_enc)).shuffle(buffer).batch(batch_size)
dev_tf_dataset_b = tf.data.Dataset.from_tensor_slices((dev_features, dev_labels_b_enc)).batch(batch_size)
test_tf_dataset_b = tf.data.Dataset.from_tensor_slices((test_features, test_labels_b_enc)).batch(batch_size)

## Model Task A

In [19]:
# initialize lists to keep statistics of all runs
f1_NAG = []
f1_CAG = []
f1_OAG = []
f1_macro = []
f1_weighted = []

# run 15 times the model to get an idea of variability
for i in range(15):

  # delete model if exists
  try:
    del BERT_model_A
  except:
    pass
  
  # define the model. Task A is a classification task with 3 labels
  BERT_model_A = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

  # compile model
  BERT_model_A.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                       loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=tf.metrics.CategoricalAccuracy()
                       )
  # fit model
  training_history = BERT_model_A.fit(train_tf_dataset_a, validation_data=dev_tf_dataset_a, epochs=2)

  print(f'---------------------------Iteration {i} ---------------------------\n')
  # Evaluate model on TEST data
  # predict using model. Returns logits
  pred_labels_test = BERT_model_A.predict(test_features)[0]
  # convert logits lo labels
  pred_labels_test = from_logits_to_labels(pred_labels_test, 'A')

  # get f1-score for all classes, macro and weighted
  x = metrics.classification_report(test_labels_a, pred_labels_test, digits=3, output_dict=True)
  # append values to keep scores
  f1_NAG.append(x['NAG']['f1-score'])
  f1_CAG.append(x['CAG']['f1-score'])
  f1_OAG.append(x['OAG']['f1-score'])
  f1_macro.append(x['macro avg']['f1-score'])
  f1_weighted.append(x['weighted avg']['f1-score'])

# calculate mean
f1_NAG_mean = round(statistics.mean(f1_NAG), 3)
f1_CAG_mean = round(statistics.mean(f1_CAG), 3)
f1_OAG_mean = round(statistics.mean(f1_OAG), 3)
f1_macro_mean = round(statistics.mean(f1_macro), 3)
f1_weighted_mean = round(statistics.mean(f1_weighted), 3)

# calculate standard deviation
f1_NAG_std = round(statistics.stdev(f1_NAG), 3)
f1_CAG_std = round(statistics.stdev(f1_CAG), 3)
f1_OAG_std = round(statistics.stdev(f1_OAG), 3)
f1_macro_std = round(statistics.stdev(f1_macro), 3)
f1_weighted_std = round(statistics.stdev(f1_weighted), 3)

print('Class NAG')
print(f'Mean f1-score = {f1_NAG_mean}')
print(f'Standard deviation f1-score = {f1_NAG_std}\n')

print('Class CAG')
print(f'Mean f1-score = {f1_CAG_mean}')
print(f'Standard deviation f1-score = {f1_CAG_std}\n')

print('Class OAG')
print(f'Mean f1-score = {f1_OAG_mean}')
print(f'Standard deviation f1-score = {f1_OAG_std}\n')

print('Class Macro')
print(f'Mean f1-score = {f1_macro_mean}')
print(f'Standard deviation f1-score = {f1_macro_std}\n')

print('Class Weighted')
print(f'Mean f1-score = {f1_weighted_mean}')
print(f'Standard deviation f1-score = {f1_weighted_std}\n')

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
586/586 [==============================] - 222s 352ms/step - loss: 0.5880 - categorical_accuracy: 0.7565 - val_loss: 0.6636 - val_categorical_accuracy: 0.7589
Epoch 2/2
586/586 [==============================] - 204s 348ms/step - loss: 0.1624 - categorical_accuracy: 0.9451 - val_loss: 0.7964 - val_categorical_accuracy: 0.7814
---------------------------Iteration 0 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
586/586 [==============================] - 219s 352ms/step - loss: 0.5917 - categorical_accuracy: 0.7516 - val_loss: 0.6723 - val_categorical_accuracy: 0.7439
Epoch 2/2
586/586 [==============================] - 204s 348ms/step - loss: 0.1512 - categorical_accuracy: 0.9480 - val_loss: 0.7136 - val_categorical_accuracy: 0.8011
---------------------------Iteration 1 ---------------------------



...

Epoch 1/2
586/586 [==============================] - 221s 352ms/step - loss: 0.5821 - categorical_accuracy: 0.7627 - val_loss: 0.6050 - val_categorical_accuracy: 0.7814
Epoch 2/2
586/586 [==============================] - 204s 348ms/step - loss: 0.1452 - categorical_accuracy: 0.9513 - val_loss: 0.7497 - val_categorical_accuracy: 0.7767
---------------------------Iteration 12 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
586/586 [==============================] - 220s 352ms/step - loss: 0.6054 - categorical_accuracy: 0.7490 - val_loss: 0.6449 - val_categorical_accuracy: 0.7542
Epoch 2/2
586/586 [==============================] - 204s 348ms/step - loss: 0.1670 - categorical_accuracy: 0.9478 - val_loss: 0.7001 - val_categorical_accuracy: 0.7777
---------------------------Iteration 13 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
586/586 [==============================] - 221s 352ms/step - loss: 0.5905 - categorical_accuracy: 0.7545 - val_loss: 0.6989 - val_categorical_accuracy: 0.7242
Epoch 2/2
586/586 [==============================] - 204s 348ms/step - loss: 0.1723 - categorical_accuracy: 0.9425 - val_loss: 0.8231 - val_categorical_accuracy: 0.7833
---------------------------Iteration 14 ---------------------------

Class NAG
Mean f1-score = 0.854
Standard deviation f1-score = 0.024

Class CAG
Mean f1-score = 0.46
Standard deviation f1-score = 0.106

Class OAG
Mean f1-score = 0.591
Standard deviation f1-score = 0.069

Class Macro
Mean f1-score = 0.635
Standard deviation f1-score = 0.043

Class Weighted
Mean f1-score = 0.718
Standard deviation f1-score = 0.033



## Model Task B

In [ ]:
# initialize lists to keep statistics of all runs
f1_NGEN = []
f1_GEN = []
f1_macro_b = []
f1_weighted_b = []

for i in range(15):

  # delete model if exists
  try:
    del BERT_model_B
  except:
    pass
  
  # define the model. Task B is a classification task with 2 labels
  BERT_model_B = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

  # compile model
  BERT_model_B.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                       loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=tf.metrics.CategoricalAccuracy()
                       )
  
  # fit model
  training_history = BERT_model_B.fit(train_tf_dataset_b, validation_data=dev_tf_dataset_b, epochs=3)

  print(f'---------------------------Iteration {i} ---------------------------\n')
  # Evaluate model on TEST data
  # predict using model. Returns logits
  pred_labels_test = BERT_model_B.predict(test_features)[0]

  # convert logits lo labels
  pred_labels_test = from_logits_to_labels(pred_labels_test, 'B')

  # get f1-score for all classes, macro and weighted
  x = metrics.classification_report(test_labels_b, pred_labels_test, digits=3, output_dict=True)
  # append values to keep scores
  f1_NGEN.append(x['NGEN']['f1-score'])
  f1_GEN.append(x['GEN']['f1-score'])
  f1_macro_b.append(x['macro avg']['f1-score'])
  f1_weighted_b.append(x['weighted avg']['f1-score'])

# calculate mean
f1_NGEN_mean = round(statistics.mean(f1_NGEN), 3)
f1_GEN_mean = round(statistics.mean(f1_GEN), 3)
f1_macro_b_mean = round(statistics.mean(f1_macro_b), 3)
f1_weighted_b_mean = round(statistics.mean(f1_weighted_b), 3)

# calculate standard deviation
f1_NGEN_std = round(statistics.stdev(f1_NGEN), 3)
f1_GEN_std = round(statistics.stdev(f1_GEN), 3)
f1_macro_b_std = round(statistics.stdev(f1_macro_b), 3)
f1_weighted_b_std = round(statistics.stdev(f1_weighted_b), 3)

print('Class NGEN')
print(f'Mean f1-score = {f1_NGEN_mean}')
print(f'Standard deviation f1-score = {f1_NGEN_std}\n')

print('Class GEN')
print(f'Mean f1-score = {f1_GEN_mean}')
print(f'Standard deviation f1-score = {f1_GEN_std}\n')

print('Macro')
print(f'Mean f1-score = {f1_macro_b_mean}')
print(f'Standard deviation f1-score = {f1_macro_b_std}\n')

print('Weighted')
print(f'Mean f1-score = {f1_weighted_b_mean}')
print(f'Standard deviation f1-score = {f1_weighted_b_std}\n')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
586/586 [==============================] - 220s 352ms/step - loss: 0.2197 - categorical_accuracy: 0.9166 - val_loss: 0.2191 - val_categorical_accuracy: 0.9428
Epoch 2/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0544 - categorical_accuracy: 0.9835 - val_loss: 0.2639 - val_categorical_accuracy: 0.9343
Epoch 3/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0288 - categorical_accuracy: 0.9900 - val_loss: 0.2487 - val_categorical_accuracy: 0.9437
---------------------------Iteration 0 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
586/586 [==============================] - 219s 352ms/step - loss: 0.3066 - categorical_accuracy: 0.8795 - val_loss: 0.2006 - val_categorical_accuracy: 0.9306
Epoch 2/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0790 - categorical_accuracy: 0.9745 - val_loss: 0.2734 - val_categorical_accuracy: 0.9259
Epoch 3/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0457 - categorical_accuracy: 0.9861 - val_loss: 0.2508 - val_categorical_accuracy: 0.9409
---------------------------Iteration 1 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/3
586/586 [==============================] - 220s 353ms/step - loss: 0.2600 - categorical_accuracy: 0.9011 - val_loss: 0.2798 - val_categorical_accuracy: 0.9184
Epoch 2/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0839 - categorical_accuracy: 0.9741 - val_loss: 0.2748 - val_categorical_accuracy: 0.9343
Epoch 3/3
586/586 [==============================] - 204s 348ms/step - loss: 0.0419 - categorical_accuracy: 0.9881 - val_loss: 0.3700 - val_categorical_accuracy: 0.9353
---------------------------Iteration 12 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
586/586 [==============================] - 220s 353ms/step - loss: 0.2243 - categorical_accuracy: 0.9169 - val_loss: 0.2137 - val_categorical_accuracy: 0.9325
Epoch 2/3
586/586 [==============================] - 203s 346ms/step - loss: 0.0554 - categorical_accuracy: 0.9830 - val_loss: 0.2571 - val_categorical_accuracy: 0.9325
Epoch 3/3
586/586 [==============================] - 203s 346ms/step - loss: 0.0268 - categorical_accuracy: 0.9907 - val_loss: 0.3062 - val_categorical_accuracy: 0.9465
---------------------------Iteration 13 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
586/586 [==============================] - 218s 350ms/step - loss: 0.2621 - categorical_accuracy: 0.9013 - val_loss: 0.2221 - val_categorical_accuracy: 0.9268
Epoch 2/3
586/586 [==============================] - 203s 346ms/step - loss: 0.0620 - categorical_accuracy: 0.9807 - val_loss: 0.2742 - val_categorical_accuracy: 0.9418
Epoch 3/3
586/586 [==============================] - 203s 347ms/step - loss: 0.0256 - categorical_accuracy: 0.9930 - val_loss: 0.3885 - val_categorical_accuracy: 0.9484
---------------------------Iteration 14 ---------------------------

Class NGEN
Mean f1-score = 0.909
Standard deviation f1-score = 0.019

Class GEN
Mean f1-score = 0.527
Standard deviation f1-score = 0.042

Macro
Mean f1-score = 0.718
Standard deviation f1-score = 0.022

Weighted
Mean f1-score = 0.854
Standard deviation f1-score = 0.017



## References

- Pre-processing data: https://huggingface.co/transformers/preprocessing.html

- Fine-tunning a pre-trained model: https://huggingface.co/transformers/training.html

- BERT: https://huggingface.co/transformers/model_doc/bert.html
